In [1]:
from network.region_proposal import RPN
import numpy as np
import tensorflow as tf

c:\users\smmeenehan\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
rpn_graph = RPN('channels_last', num_channels=32, anchors_per_loc=4)

In [3]:
rand_tens = tf.convert_to_tensor(np.random.randn(10, 7, 7, 64).astype('float32'))

In [5]:
with tf.Session() as sess:
    [logits, probs, bbox] = rpn_graph(rand_tens)
    sess.run(tf.global_variables_initializer())
    print(tf.shape(logits).eval())

[ 10 196   2]


In [3]:
from backbone import ResNet
import matplotlib.pyplot as plt
import numpy as np
from region_proposal import RPN, ProposalLayer
from resnet_var_dict import resnet_var_dict
from roi_classifier import ROIAlign
import skimage
from skimage import io, transform
import tensorflow as tf
from utils import generate_anchors

In [4]:
tf.enable_eager_execution()

In [3]:
backbone = ResNet('channels_last', num_blocks=[3, 4, 6, 3], include_fc=False)
rpn_graph = RPN('channels_last')
proposer = ProposalLayer()

In [4]:
real_picture_1 = io.imread('./example_data/imagenet_tabby_cat_1.jpg').astype('float32')
real_picture_2 = io.imread('./example_data/imagenet_tabby_cat_3.jpg').astype('float32')
input_data = tf.convert_to_tensor(np.stack([real_picture_1, real_picture_2]))

In [5]:
feature_maps = backbone(input_data)
rpn_logits, rpn_probs, rpn_boxes = rpn_graph(feature_maps)

In [54]:
anchors = generate_anchors([32, 64, 128], [0.5, 1, 1.5], 
                           tf.shape(input_data)[1:3],
                           tf.shape(feature_maps)[1:3], 1)
anchors = tf.tile(tf.expand_dims(anchors,0), [tf.shape(input_data)[0], 1, 1])

In [96]:
rois = proposer([rpn_probs, rpn_boxes, anchors])

In [112]:
roi_aligner = ROIAlign()

In [122]:
roi_features = roi_aligner([feature_maps, rois])

In [116]:
roi_features

<tf.Tensor: id=7952, shape=(2, 2000, 7, 7, 2048), dtype=float32, numpy=
array([[[[[ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.613046  ,
            4.122717  ,  0.        ],
          [ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.613046  ,
            4.122717  ,  0.        ],
          [ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.613046  ,
            4.122717  ,  0.        ],
          ...,
          [ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.613046  ,
            4.122717  ,  0.        ],
          [ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.613046  ,
            4.122717  ,  0.        ],
          [ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.613046  ,
            4.122717  ,  0.        ]],

         [[ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.613046  ,
            4.122717  ,  0.        ],
          [ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.613046  ,
            4.122717  ,  0.        ],
          [ 2.5480297 ,  8.164041  ,  1.1206939 , ...,  6.61304